<a href="https://colab.research.google.com/github/singharshika/movie-recommender-system/blob/main/MOVIE_RECOMMNEDER_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
#loading the data from the csv file to a pandas dataframe
#loading the data from the csv file to a pandas dataframe
movies_data= pd.read_csv('/content/movies (1).csv', on_bad_lines='skip') # Use 'on_bad_lines' to handle bad lines

# Print rows with errors, this will now be empty since bad lines are skipped
print(movies_data[movies_data['title'].isnull()])

Empty DataFrame
Columns: [index, budget, genres, homepage, id, keywords, original_language, original_title, overview, popularity, production_companies, production_countries, release_date, revenue, runtime, spoken_languages, status, tagline, title, vote_average, vote_count, cast, crew, director]
Index: []

[0 rows x 24 columns]


In [ ]:
movies_data.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [ ]:
#number of row and columns in the data frame
movies_data.shape

(4803, 24)

In [ ]:
selected_features = ['genres','keywords','tagline','cast','director']
print(selected_features)

['genres', 'keywords', 'tagline', 'cast', 'director']


In [ ]:
#replacing the null value with the null string
for feature in selected_features:
    movies_data[feature] =  movies_data[feature].fillna('')

# Combine features, handling potential NaN values
combined_features = movies_data[selected_features].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1)

# Verify if there are still any NaN values
print(combined_features[combined_features.isna()])

# Now proceed with vectorization
vectorizer= TfidfVectorizer()
feature_vectors= vectorizer.fit_transform(combined_features)

Series([], dtype: object)


In [ ]:
print(feature_vectors)

  (0, 2432)	0.17272411194153
  (0, 7755)	0.1128035714854756
  (0, 13024)	0.1942362060108871
  (0, 10229)	0.16058685400095302
  (0, 8756)	0.22709015857011816
  (0, 14608)	0.15150672398763912
  (0, 16668)	0.19843263965100372
  (0, 14064)	0.20596090415084142
  (0, 13319)	0.2177470539412484
  (0, 17290)	0.20197912553916567
  (0, 17007)	0.23643326319898797
  (0, 13349)	0.15021264094167086
  (0, 11503)	0.27211310056983656
  (0, 11192)	0.09049319826481456
  (0, 16998)	0.1282126322850579
  (0, 15261)	0.07095833561276566
  (0, 4945)	0.24025852494110758
  (0, 14271)	0.21392179219912877
  (0, 3225)	0.24960162956997736
  (0, 16587)	0.12549432354918996
  (0, 14378)	0.33962752210959823
  (0, 5836)	0.1646750903586285
  (0, 3065)	0.22208377802661425
  (0, 3678)	0.21392179219912877
  (0, 5437)	0.1036413987316636
  :	:
  (4801, 17266)	0.2886098184932947
  (4801, 4835)	0.24713765026963996
  (4801, 403)	0.17727585190343226
  (4801, 6935)	0.2886098184932947
  (4801, 11663)	0.21557500762727902
  (4801, 1672

In [ ]:
similarity= cosine_similarity(feature_vectors)

In [ ]:
print(similarity)

[[1.         0.07219487 0.037733   ... 0.         0.         0.        ]
 [0.07219487 1.         0.03281499 ... 0.03575545 0.         0.        ]
 [0.037733   0.03281499 1.         ... 0.         0.05389661 0.        ]
 ...
 [0.         0.03575545 0.         ... 1.         0.         0.02651502]
 [0.         0.         0.05389661 ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.02651502 0.         1.        ]]


In [ ]:
print(similarity.shape)

(4803, 4803)


In [ ]:
# getting the movie name from the user

movie_name = input('Enter your favourite movie name :')

In [ ]:
# creating a list with all the movie names given in the dataset
list_of_all_titles = movies_data['title'].tolist()
print(list_of_all_titles)

In [ ]:
#finding the close match for the movie name given by the user
find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)
print(find_close_match)

In [ ]:
close_match= find_close_match[0]
print(close_match)

In [ ]:
#finding index of the movie
index_of_the_movie = movies_data[movies_data.title == close_match]['index'].values[0]
print(index_of_the_movie)

In [ ]:
#getting a list of similar movies
similarity_score = list(enumerate(similarity[index_of_the_movie]))
print(similarity_score)

In [ ]:
len(similarity_score)

In [ ]:
sorted_similar_movies = sorted(similarity_score,key=lambda x:x[1],reverse=True)
print(sorted_similar_movies)

In [ ]:
# print the names of similar movies based on the index
print('Movies suggested for you : \n')

i =1

for movie in sorted_similar_movies:
    index = movie[0]
    title_from_index = movies_data[movies_data.index==index]['title'].values[0]
    if(i<30):
        print(i,'.',title_from_index) # Indent this line to be part of the if block
        i+=1 # Also indent this line to be part of the if block

In [ ]:
movie_name  = input('Enter your favourite movie name : ')
list_of_all_titles = movies_data['title'].tolist()
find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)
close_match= find_close_match[0]
index_of_the_movie = movies_data[movies_data.title == close_match]['index'].values[0]
similarity_score = list(enumerate(similarity[index_of_the_movie]))
sorted_similar_movies = sorted(similarity_score,key=lambda x:x[1],reverse=True)
print('Movies suggested for you : \n')

i =1

for movie in sorted_similar_movies:
    index = movie[0]
    title_from_index = movies_data[movies_data.index==index]['title'].values[0]
    if(i<30):
        print(i,'.',title_from_index)
        i+=1

In [ ]:
!pip install gradio

In [ ]:
import pandas as pd
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import gradio as gr
import requests

# Load the data from the CSV file to a pandas dataframe
movies_data = pd.read_csv('/content/movies (1).csv', on_bad_lines='skip')

# Select features
selected_features = ['genres', 'keywords', 'tagline', 'cast', 'director']

# Replace null values with empty strings
for feature in selected_features:
    movies_data[feature] = movies_data[feature].fillna('')

# Combine selected features into a single string
combined_features = movies_data['genres'] + ' ' + movies_data['keywords'] + ' ' + movies_data['tagline'] + ' ' + movies_data['cast'] + ' ' + movies_data['director']

# Vectorize the features
vectorizer = TfidfVectorizer()
feature_vectors = vectorizer.fit_transform(combined_features)

# Compute cosine similarity
similarity = cosine_similarity(feature_vectors)

# Function to fetch movie poster
def fetch_poster(movie_id):
    url = f"https://api.themoviedb.org/3/movie/{movie_id}?api_key=c7ec19ffdd3279641fb606d19ceb9bb1&language=en-US"
    data = requests.get(url).json()
    poster_path = data.get('poster_path')
    if poster_path:
        full_path = f"https://image.tmdb.org/t/p/w500/{poster_path}"
        return full_path
    else:
        return "https://via.placeholder.com/500x750?text=No+Image"

# Function to get movie recommendations
def recommend(movie_name):
    list_of_all_titles = movies_data['title'].tolist()
    find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)
    close_match = find_close_match[0]
    index_of_the_movie = movies_data[movies_data.title == close_match]['index'].values[0]
    similarity_score = list(enumerate(similarity[index_of_the_movie]))
    sorted_similar_movies = sorted(similarity_score, key=lambda x: x[1], reverse=True)[1:11]

    recommended_movie_names = []
    recommended_movie_posters = []
    for movie in sorted_similar_movies:
        index = movie[0]
        recommended_movie_names.append(movies_data.iloc[index]['title'])
        movie_id = movies_data.iloc[index]['id']
        recommended_movie_posters.append(fetch_poster(movie_id))

    return recommended_movie_names, recommended_movie_posters

# Gradio Blocks interface
with gr.Blocks() as demo:
    gr.Markdown("# Movie Recommender System")
    with gr.Row():
        selectvalue = gr.Dropdown(label="Select movie from dropdown", choices=movies_data['title'].tolist(), value=movies_data['title'].tolist()[0])
    show_button = gr.Button("Show Recommend")

    with gr.Row():
        col1 = gr.Column(scale=1)
        col2 = gr.Column(scale=1)
        col3 = gr.Column(scale=1)
        col4 = gr.Column(scale=1)
        col5 = gr.Column(scale=1)

    with col1:
        rec1_name = gr.Textbox(label="Movie 1")
        rec1_img = gr.Image()
    with col2:
        rec2_name = gr.Textbox(label="Movie 2")
        rec2_img = gr.Image()
    with col3:
        rec3_name = gr.Textbox(label="Movie 3")
        rec3_img = gr.Image()
    with col4:
        rec4_name = gr.Textbox(label="Movie 4")
        rec4_img = gr.Image()
    with col5:
        rec5_name = gr.Textbox(label="Movie 5")
        rec5_img = gr.Image()

    with gr.Row():
        col6 = gr.Column(scale=1)
        col7 = gr.Column(scale=1)
        col8 = gr.Column(scale=1)
        col9 = gr.Column(scale=1)
        col10 = gr.Column(scale=1)

    with col6:
        rec6_name = gr.Textbox(label="Movie 6")
        rec6_img = gr.Image()
    with col7:
        rec7_name = gr.Textbox(label="Movie 7")
        rec7_img = gr.Image()
    with col8:
        rec8_name = gr.Textbox(label="Movie 8")
        rec8_img = gr.Image()
    with col9:
        rec9_name = gr.Textbox(label="Movie 9")
        rec9_img = gr.Image()
    with col10:
        rec10_name = gr.Textbox(label="Movie 10")
        rec10_img = gr.Image()

    def update_recommendations(selectvalue):
        movie_names, movie_posters = recommend(selectvalue)
        return (
            [movie_names[0], movie_posters[0]], [movie_names[1], movie_posters[1]], [movie_names[2], movie_posters[2]],
            [movie_names[3], movie_posters[3]], [movie_names[4], movie_posters[4]], [movie_names[5], movie_posters[5]],
            [movie_names[6], movie_posters[6]], [movie_names[7], movie_posters[7]], [movie_names[8], movie_posters[8]],
            [movie_names[9], movie_posters[9]]
        )

    show_button.click(
        update_recommendations,
        inputs=selectvalue,
        outputs=[
            rec1_name, rec1_img, rec2_name, rec2_img, rec3_name, rec3_img, rec4_name, rec4_img, rec5_name, rec5_img,
            rec6_name, rec6_img, rec7_name, rec7_img, rec8_name, rec8_img, rec9_name, rec9_img, rec10_name, rec10_img
        ]
    )

demo.launch()
